In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import csv
import os

In [2]:
baidureader = pd.read_csv("data/Baidu Mobility Data/Jan/baidu_in_20200101.csv")
print(baidureader)

       city_name   北京市    天津市     河北     山西   内蒙古    辽宁    吉林   黑龙江   上海市  \
0            北京市   NaN  17.46  41.12  11.05  9.59  9.71  6.82  9.61  2.20   
1            天津市  7.79    NaN  17.51   3.35  3.01  3.87  3.10  5.27  0.55   
2             河北   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
3             山西   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
4            内蒙古   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
5             辽宁   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
6             吉林   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
7            黑龙江   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
8            上海市  1.62   0.83   0.58   1.41  0.75  2.30  1.79  2.18   NaN   
9             江苏   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
10            浙江   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   
11            安徽   NaN    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   

In [3]:
mobInCity = np.zeros((339,339,116)).astype("float")
mobOutCity = np.zeros((339,339,116)).astype("float")
provinceIdx = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]

In [4]:
#in processing
#rearrange data for Yunnan before 3/9
def yunnan(result):
    resultc = result.copy()
    baoshanr = resultc[296,:]
    baoshanc = resultc[:,296]
    zhaotongr = resultc[297,:]
    zhaotongc = resultc[:,297]
    lijiangr = resultc[298,:]
    lijiangc = resultc[:,298]
    puerr = resultc[299,:]
    puerc = resultc[:,299]
    lincangr = resultc[300,:]
    lincangc = resultc[:,300]
    chuxiongr = resultc[301,:]
    chuxiongc = resultc[:,301]
    hongher = resultc[302,:]
    honghec = resultc[:,302]
    wenshanr = resultc[303,:]
    wenshanc = resultc[:,303]
    xishuangbannar = resultc[304,:]
    xishuangbannac = resultc[:,304]
    dalir = resultc[305,:]
    dalic = resultc[:,305]
    dehongr = resultc[306,:]
    dehongc = resultc[:,306]
    nujiangr = resultc[307,:]
    nujiangc = resultc[:,307]
    diqingr = resultc[308,:]
    diqingc = resultc[:,308]
    result[296,:] = zhaotongr
    result[:,296] = zhaotongc
    result[297,:] = chuxiongr
    result[:,297] = chuxiongc
    result[298,:] = hongher
    result[:,298] = honghec
    result[299,:] = wenshanr
    result[:,299] = wenshanc
    result[300,:] = puerr
    result[:,300] = puerc
    result[301,:] = xishuangbannar
    result[:,301] = xishuangbannac
    result[302,:] = dalir
    result[:,302] = dalic
    result[303,:] = baoshanr
    result[:,303] = baoshanc
    result[304,:] = dehongr
    result[:,304] = dehongc
    result[305,:] = lijiangr
    result[:,305] = lijiangc
    result[306,:] = nujiangr
    result[:,306] = nujiangc
    result[307,:] = diqingr
    result[:,307] = diqingc
    result[308,:] = lincangr
    result[:,308] = lincangc
    return result

#should delete row 195
#should change row/col 234 to data for chongqing and delete row/col 3
def baiduInProcess(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = yunnan(result)
    #delete row/col 195
    result = np.delete(result,195,0)
    result = np.delete(result,195,1)
    #delete row/col 21 & change row/col 262
    CQR = result[21,:].copy()
    CRC = result[:,21].copy()
    result[261,:] = CQR
    result[:,261] = CRC
    result = np.delete(result,21,0)
    result = np.delete(result,21,1)
    #put Shanghai in the right order
    SHR = result[8,:].copy()
    result = np.insert(result,100,SHR,axis = 0)
    result = np.delete(result,8,0)
    SHC = result[:,8].copy()
    result = np.insert(result,100,SHC,axis = 1)
    result = np.delete(result,8,1)
    rows,cols = result.shape
    nc = 0
    nr = 0
    for c in range(cols):
        if c not in provinceIdx:
            nr = 0
            for r in range(rows):
                if r not in provinceIdx:
                    mobInCity[nr,nc,date] = result[r,c]
                    nr += 1
            nc += 1

#remove Chaohu and insert Zhongwei
def BaiduInProcess39(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = np.delete(result,109,0)
    result = np.delete(result,109,1)
    zwr = [0]*result.shape[0]
    result = np.insert(result,324,zwr,0)
    zwc = [0]*result.shape[0]
    result = np.insert(result,324,zwc,1)
    rows,cols = result.shape
    mobInCity[:,:,date] = result
                    
date = 0
Jan = os.fsencode("data/Baidu Mobility Data/Jan")
for filename in sorted(os.listdir(Jan)):
    filestr = filename.decode(encoding="utf-8")
    if "in" in filestr:
        address = "data/Baidu Mobility Data/Jan/"+filestr
        baiduInProcess(address,date)
        date += 1
        
Feb = os.fsencode("data/Baidu Mobility Data/Feb")
for filename in sorted(os.listdir(Feb)):
    filestr = filename.decode(encoding="utf-8")
    if "in" in filestr:
        address = "data/Baidu Mobility Data/Feb/"+filestr
        baiduInProcess(address,date)
        date += 1

MA = os.fsencode("data/Baidu Mobility Data")
idx = 0
for filename in sorted(os.listdir(MA)):
    filestr = filename.decode(encoding="utf-8")
    if "in" in filestr and idx < 8:
        address = "data/Baidu Mobility Data/"+filestr
        baiduInProcess(address,date)
        date += 1
        idx += 1
    elif "in" in filestr and idx >= 8 and date < 95:
        address = "data/Baidu Mobility Data/"+filestr
        BaiduInProcess39(address,date)
        date += 1
        idx += 1


In [5]:
'''
Process data after 4/5/2020 by expanding the shape of mobInCity to (349,349)
'''
mobInCityn = np.zeros((349,349,116))
mobInCityn[:339,:339,:95] = mobInCity[:,:,:95]

def baiduInProcess45(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = np.delete(result,109,0)
    result = np.delete(result,109,1)
    zwr = [0]*result.shape[0]
    result = np.insert(result,324,zwr,0)
    zwc = [0]*result.shape[0]
    result = np.insert(result,324,zwc,1)
    rows,cols = result.shape
    mobInCityn[:,:,date] = result

date = 95
MA = os.fsencode("data/Baidu Mobility Data")
for filename in sorted(os.listdir(MA)):
    filestr = filename.decode(encoding="utf-8")
    if "in" in filestr and int(filestr[-7]) == 4 and (int(filestr[-5]) > 4 or int(filestr[-6:-4]) >= 10):
        address = "data/Baidu Mobility Data/"+filestr
        baiduInProcess45(address,date)
        date += 1

In [6]:
print(mobInCityn[:,72,0])
print(mobInCityn[:,72,-1])

[ 2.2   0.55  0.21   nan   nan   nan   nan  0.1    nan   nan   nan   nan
   nan  0.16   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  0.29  0.27   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan  0.22  0.09   nan   nan   nan   nan   nan   nan   nan  0.32
   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan  2.39  3.55  0.86  1.82 24.27  5.17  0.44  0.64  2.06  1.04  0.77
  1.44  0.64  4.81  2.36  0.94  6.03  1.26  1.25  0.98  0.2   1.16  0.73
  0.16  1.58  0.74  0.54  0.54  0.25  0.21  0.16  0.36  0.34  0.58  0.76
  0.41  0.64  0.41  0.19  0.74  0.35  0.51   nan   nan  0.16   nan  0.13
   nan  0.21  0.47  0.13   nan  0.21   nan   nan  0.12  0.1   0.15  0.19
  0.49  0.31  0.47  0.1   0.24   nan  0.2   0.21  0.24   nan   nan   nan
   nan  0.47   nan   nan   nan  0.27  0.67  0.13  0.15   nan   nan   nan
  0.11   nan   nan   nan   nan   nan  0.17  0.34  0

In [7]:
trans = pd.read_csv("data/Baidu Mobility Data/BaiduMobilityDirectory.csv")
print(trans)
cityEN = trans["City_EN"]

                            City_EN      City_CH Prov_CH    Prov_EN
0                    Beijing_110000          北京市     北京市    Beijing
1                    Tianjin_120000          天津市     天津市    Tianjin
2               Shijiazhuang_130100         石家庄市     河北省      Hebei
3                   Tangshan_130200          唐山市     河北省      Hebei
4                Qinhuangdao_130300         秦皇岛市     河北省      Hebei
5                     Handan_130400          邯郸市     河北省      Hebei
6                    Xingtai_130500          邢台市     河北省      Hebei
7                    Baoding_130600          保定市     河北省      Hebei
8                Zhangjiakou_130700         张家口市     河北省      Hebei
9                    Chengde_130800          承德市     河北省      Hebei
10                  Cangzhou_130900          沧州市     河北省      Hebei
11                  Langfang_131000          廊坊市     河北省      Hebei
12                  Hengshui_131100          衡水市     河北省      Hebei
13                   Taiyuan_140100          太原市

In [8]:
#create a csv file for in_data
import datetime as dt
filename = "baidu_in_clean.csv"
fields = ["Date","City_in","City_out","%"]
dayJan = 1
dayFeb = 1
dayMar = 1
dayApr = 1
rows = []
#0-30 Jan, 31-59 Feb, 60-90 Mar, 91-115 Apr

rows,cols,times = mobInCityn.shape
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields) 
    for t in range(times):
        if t < 31:
            d = dt.datetime(2020,1,dayJan)
            day = d.strftime("%x")
            dayJan += 1
        elif t < 60:
            d = dt.datetime(2020,2,dayFeb)
            day = d.strftime("%x")
            dayFeb += 1
        elif t < 91:
            d = dt.datetime(2020,3,dayMar)
            day = d.strftime("%x")
            dayMar += 1
        else:
            d = dt.datetime(2020,4,dayApr)
            day = d.strftime("%x")
            dayApr += 1
            if t == 115:
                print("complete!")
        for c in range(cols):
            ci = cityEN[c]
            for r in range(rows):
                co = cityEN[r]
                csvwriter.writerow([day,ci,co,mobInCityn[r,c,t]])

complete!


In [9]:
#Out processing
    
#should delete row 195
#should change row/col 234 to data for chongqing and delete row/col 3
def baiduOutProcess(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = yunnan(result)
    #delete row/col 195
    result = np.delete(result,195,0)
    result = np.delete(result,195,1)
    #delete row/col 21 & change row/col 262
    CQR = result[21,:].copy()
    CRC = result[:,21].copy()
    result[261,:] = CQR
    result[:,261] = CRC
    result = np.delete(result,21,0)
    result = np.delete(result,21,1)
    #put Shanghai in the right order
    SHR = result[8,:].copy()
    result = np.insert(result,100,SHR,axis = 0)
    result = np.delete(result,8,0)
    SHC = result[:,8].copy()
    result = np.insert(result,100,SHC,axis = 1)
    result = np.delete(result,8,1)
    rows,cols = result.shape
    nc = 0
    nr = 0
    for c in range(cols):
        if c not in provinceIdx:
            nr = 0
            for r in range(rows):
                if r not in provinceIdx:
                    mobOutCity[nr,nc,date] = result[r,c]
                    nr += 1
            nc += 1

#remove Chaohu and insert Zhongwei
def BaiduOutProcess39(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = np.delete(result,109,0)
    result = np.delete(result,109,1)
    zwr = [0]*result.shape[0]
    result = np.insert(result,324,zwr,0)
    zwc = [0]*result.shape[0]
    result = np.insert(result,324,zwc,1)
    rows,cols = result.shape
    mobOutCity[:,:,date] = result
                    
date = 0
Jan = os.fsencode("data/Baidu Mobility Data/Jan")
for filename in sorted(os.listdir(Jan)):
    filestr = filename.decode(encoding="utf-8")
    if "out" in filestr:
        address = "data/Baidu Mobility Data/Jan/"+filestr
        baiduOutProcess(address,date)
        date += 1
        
Feb = os.fsencode("data/Baidu Mobility Data/Feb")
for filename in sorted(os.listdir(Feb)):
    filestr = filename.decode(encoding="utf-8")
    if "out" in filestr:
        address = "data/Baidu Mobility Data/Feb/"+filestr
        baiduOutProcess(address,date)
        date += 1

MA = os.fsencode("data/Baidu Mobility Data")
idx = 0
for filename in sorted(os.listdir(MA)):
    filestr = filename.decode(encoding="utf-8")
    #exception
    if filestr == "baidu_out_20200322.csv":
        date += 1
        idx += 1
        continue
    elif "out" in filestr and idx < 8:
        address = "data/Baidu Mobility Data/"+filestr
        baiduOutProcess(address,date)
        date += 1
        idx += 1
    elif "out" in filestr and idx >= 8 and date < 95:
        address = "data/Baidu Mobility Data/"+filestr
        BaiduOutProcess39(address,date)
        date += 1
        idx += 1

'''
Process data after 4/5/2020 by expanding the shape of mobOutCity to (349,349)
'''
mobOutCityn = np.zeros((349,349,116))
mobOutCityn[:339,:339,:95] = mobOutCity[:,:,:95]

def baiduOutProcess45(file,date):
    reader = pd.read_csv(file)
    result = reader.values[:,1:]
    result = np.delete(result,109,0)
    result = np.delete(result,109,1)
    zwr = [0]*result.shape[0]
    result = np.insert(result,324,zwr,0)
    zwc = [0]*result.shape[0]
    result = np.insert(result,324,zwc,1)
    rows,cols = result.shape
    mobOutCityn[:,:,date] = result

date = 95
MA = os.fsencode("data/Baidu Mobility Data")
for filename in sorted(os.listdir(MA)):
    filestr = filename.decode(encoding="utf-8")
    if filestr == "baidu_out_20200322.csv":
        address = "data/Baidu Mobility Data/"+filestr
        baiduOutProcess45(address,81)
    if "out" in filestr and int(filestr[-7]) == 4 and (int(filestr[-5]) > 4 or int(filestr[-6:-4]) >= 10):
        address = "data/Baidu Mobility Data/"+filestr
        baiduOutProcess45(address,date)
        date += 1

In [10]:
print(mobOutCityn[:,:,-1])

[[  nan 11.81  3.6  ...   nan  0.44  0.89]
 [10.7    nan  2.62 ...   nan  0.13  0.07]
 [ 2.18  1.87   nan ...   nan  0.03  0.23]
 ...
 [  nan   nan   nan ...   nan  1.56  1.92]
 [  nan   nan   nan ...   nan   nan  2.61]
 [  nan   nan   nan ...   nan  1.67   nan]]


In [11]:
#create csv for out
filename = "baidu_out_clean.csv"
fields = ["Date","City_out","City_in","%"]
dayJan = 1
dayFeb = 1
dayMar = 1
dayApr = 1
rows = []
#0-30 Jan, 31-59 Feb, 60-90 Mar, 91-115 Apr

rows,cols,times = mobOutCityn.shape
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields) 
    for t in range(times):
        if t < 31:
            d = dt.datetime(2020,1,dayJan)
            day = d.strftime("%x")
            dayJan += 1
        elif t < 60:
            d = dt.datetime(2020,2,dayFeb)
            day = d.strftime("%x")
            dayFeb += 1
        elif t < 91:
            d = dt.datetime(2020,3,dayMar)
            day = d.strftime("%x")
            dayMar += 1
        else:
            d = dt.datetime(2020,4,dayApr)
            day = d.strftime("%x")
            dayApr += 1
            if t == 115:
                print("complete!")
        for c in range(cols):
            co = cityEN[c]
            for r in range(rows):
                ci = cityEN[r]
                csvwriter.writerow([day,co,ci,mobOutCityn[r,c,t]])

complete!
